In [20]:
import numpy as np
import urllib2
import csv
import pandas as pd
from subprocess import call
from scipy.interpolate import UnivariateSpline
import matplotlib.pyplot as plt


mean = 0.0
std = 0.0
# initialYear = int(input("Enter initialYear: "))
# endyear = int(input("Enter endyear: "))
initialYear = 2001
endyear = 2004
years = endyear - initialYear + 1

city = "toronto"
code = "CYTZ"
code

'CYTZ'

In [21]:
def makeCSV(year):
    csvfile = urllib2.urlopen('http://www.wunderground.com/history/airport/'+code+'/'+str(year)+'/1/1/CustomHistory.html?dayend=31&monthend=12&yearend=2015&req_city=&req_state=&req_statename=&reqdb.zip=&reqdb.magic=&reqdb.wmo=&format=1')
    csvResponse = csvfile.read()
    target = open("temp.csv", 'w')
    target.write(csvResponse)


    mainRows = []
    with open('temp.csv') as csvfile:
        csvfile = csv.reader(csvfile)
        for row_no,row in enumerate(csvfile):
            if row_no > 0:
                mainRows.append(row)
    
    call(["rm","temp.csv"])

    filename = city+str(year)+'.csv'
    with open(filename, 'w') as fp:
        a = csv.writer(fp, delimiter=',')
        a.writerows(mainRows)
            
def getFiveYearsMean(year):
    filename = city+year+'.csv'
    data = pd.read_csv(filename)
    global mean
    global std

    meanTempDay = data[data.columns[2]]
    mean =mean + round(meanTempDay.mean(),2)/years
    std = std + round(meanTempDay.std(),2)/years
    # maxT = meanTempDay.max()

    l = []
    for i in xrange(24):
        l.append( (meanTempDay[15*i:15*i+15].mean() ) )

    return pd.DataFrame(l)/years


def getBSplineCurve(fifteenDaysMean):
    x = np.linspace(0,23,24)
    y = np.array(fifteenDaysMean)
    plt.plot(x, y,'ro', ms= 5)
    plt.xlabel('15 days mean')
    plt.ylabel('temperature')
    plt.title(city+str(initialYear)+'-'+str(endyear))

    spl = UnivariateSpline(x, y)
    xs = np.linspace(0, 23, 1000)
    plt.plot(xs, spl(xs), 'g', lw=3)

    plt.text(1,fifteenDaysMean.mean()*1.5,'mu = '+ str(mean)+' sigma = '+ str(std), horizontalalignment='left',
     verticalalignment='center')
    plt.savefig(city+str(initialYear)+'.png')
    plt.show()
    

In [22]:
for curYear in range(initialYear,endyear+1):
    makeCSV(curYear)
    

In [23]:
fifteenDaysMean = None
for curYear in range(initialYear,endyear+1):
    if fifteenDaysMean is None:
        fifteenDaysMean = getFiveYearsMean(str(curYear))
    else:
        fifteenDaysMean = fifteenDaysMean + getFiveYearsMean(str(curYear))

In [24]:
fifteenDaysMean

,0
0,-3.150000
1,-5.266667
2,-2.933333
3,-2.516667
4,-1.050000
5,2.166667
6,4.200000
7,7.783333
8,10.916667
9,11.666667


In [25]:
getBSplineCurve(fifteenDaysMean)